# Import Library

In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 8, 5
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv('PM25_train.csv')
data.head()


,device_id,Date,Time,PM2.5,PM10,PM1,Temperature,Humidity,lat,lon
0,28C2DDDD415C,2017-01-01,08:03:09,21.0,0.0,0.0,24.12,83.0,23.741,120.755
1,28C2DDDD415C,2017-01-01,08:03:09,21.0,0.0,0.0,24.12,83.0,23.741,120.755
2,28C2DDDD415C,2017-01-01,08:03:09,21.0,0.0,0.0,24.12,83.0,23.741,120.755
3,28C2DDDD415C,2017-01-01,08:09:04,20.0,0.0,0.0,24.12,82.0,23.741,120.755
4,28C2DDDD415C,2017-01-01,08:09:04,20.0,0.0,0.0,24.12,82.0,23.741,120.755


# EDA

In [3]:
gPM25 = data.groupby(['device_id'])['PM2.5'].mean()
gPM25.head()

device_id
28C2DDDD415C    34.140247
28C2DDDD47BC    39.766015
74DA3895C1F8    44.860509
74DA3895C1FA    38.921148
74DA3895C200    46.241431
Name: PM2.5, dtype: float64

In [4]:
gPM25 = data.groupby(['device_id'])['PM2.5'].mean().reset_index()
gPM25 = gPM25.set_index('device_id').T.to_dict('list')

gPM10 = data.groupby(['device_id'])['PM10'].mean().reset_index()
gPM10 = gPM10.set_index('device_id').T.to_dict('list')

gPM1 = data.groupby(['device_id'])['PM1'].mean().reset_index()
gPM1 = gPM1.set_index('device_id').T.to_dict('list')

PM25_MSE = 38.018157



In [5]:
data["PM2.5"].describe()


count    1.116554e+06
mean     3.818598e+01
std      1.747436e+01
min      0.000000e+00
25%      2.700000e+01
50%      3.800000e+01
75%      4.800000e+01
max      7.860000e+02
Name: PM2.5, dtype: float64

In [6]:

#plt.plot(X['Date'],X['PM2.5'],'.')
#plt.plot(data['lat'],data['lon'],'.')

In [7]:
# 
TS_WEIGHT = 1000000000
data["Timestamp"] =pd.to_datetime(data["Date"].astype(str) + " " + data["Time"], format='%Y-%m-%d %H:%M:%S').astype(np.int64) 
#data["Timestamp"] =pd.to_datetime(data["Time"], format='%H:%M:%S').astype(np.int64) 
data["Timestamp"] = data[ "Timestamp"] // TS_WEIGHT - 1480000000
#"2017-01-01 08:03:09" => 1483257789.0
#"2017-01-01 08:09:04" => 1483258144.0
data.head()

,device_id,Date,Time,PM2.5,PM10,PM1,Temperature,Humidity,lat,lon,Timestamp
0,28C2DDDD415C,2017-01-01,08:03:09,21.0,0.0,0.0,24.12,83.0,23.741,120.755,3257789
1,28C2DDDD415C,2017-01-01,08:03:09,21.0,0.0,0.0,24.12,83.0,23.741,120.755,3257789
2,28C2DDDD415C,2017-01-01,08:03:09,21.0,0.0,0.0,24.12,83.0,23.741,120.755,3257789
3,28C2DDDD415C,2017-01-01,08:09:04,20.0,0.0,0.0,24.12,82.0,23.741,120.755,3258144
4,28C2DDDD415C,2017-01-01,08:09:04,20.0,0.0,0.0,24.12,82.0,23.741,120.755,3258144


In [8]:
#1.53438595694/0.13918
def removeMissing(df):
    df = df.drop(df[df["PM10"]  == 0].index)
    df = df.drop(df[df["PM1"]  == 0].index)
    return df
    #data = addMissing(data)

#PM10及PM1有缺值 經觀察
#PM10約PM25 +3~4,#PM1約PM25-(6~7)
#補完mse只有1.59 ，但..... test data沒有pm2可以抓... 更，只能塞mse
def addMissing(df):
    df.loc[df["PM10"]  == 0 ,"PM10" ]  = df["PM2.5"] + 3.5
    df.loc[df["PM1"]  == 0 ,"PM1" ]  = df["PM2.5"] - 6.5
    return df
def addMissingMse(df):
    df.loc[df["PM10"]  == 0 ,"PM10" ]  = PM25_MSE + 3.5
    df.loc[df["PM1"]  == 0 ,"PM1" ]  = PM25_MSE - 6.5
    return df
def addMissingMean(df):
    for i, row in df.iterrows():
        if row["PM10"]  == 0:
            #df.at[i,'PM10'] = gPM25[row["device_id"]][0] + gPM10[row["device_id"]][0]
            df.at[i,'PM10'] = gPM25[row["device_id"]][0] + 3.5 #gPM10[row["device_id"]][0]
        if row["PM1"]  == 0:
            df.at[i,'PM1'] = gPM25[row["device_id"]][0] - 6.5 # gPM1[row["device_id"]][0]
    return df  


In [9]:
#handle missing data
data = removeMissing(data)
data.head()
data.shape

(1103807, 11)

# X,y


In [10]:

X = data.drop(['device_id','Date','Time','PM2.5'],axis=1)
X.head()

,PM10,PM1,Temperature,Humidity,lat,lon,Timestamp
10961,61.0,34.0,22.62,81.0,24.149,120.681,3257897
10962,61.0,34.0,22.62,81.0,24.149,120.681,3257897
10963,61.0,34.0,22.62,81.0,24.149,120.681,3257897
10964,62.0,35.0,22.75,81.0,24.149,120.681,3258252
10965,62.0,35.0,22.75,81.0,24.149,120.681,3258252


In [11]:
y = data["PM2.5"].as_matrix()
y.shape

(1103807,)

# model

In [12]:
from sklearn import linear_model

linear = linear_model.LinearRegression(normalize=True) #Score: 0.994944403792355 MSE: 1.5351809358318766

ard = linear_model.ARDRegression()

alpha =  1e-6

ridge = linear_model.Ridge(alpha = alpha, normalize=True) # Score: 0.9949444037920316 MSE: 1.535180935930128 (alpha =  1e-6)

#lasso
lasso = linear_model.Lasso(alpha = alpha, normalize=True) #Score: 0.9948977875106408 MSE: 1.5493364229491262
lars = linear_model.LassoLars() #305
#linear_model.LarsCV() #255.522

reg = linear

reg.fit (X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [13]:
y_pred = reg.predict(X)
mse = mean_squared_error(y_pred,y)
print (reg.coef_)
print("Score: {} MSE: {}".format(reg.score(X,y),mse))

[  3.72651530e-01   7.60921105e-01   1.81132639e-04  -1.06380937e-03
   5.75388403e-01  -8.26605424e-02   1.23954613e-08]
Score: 0.994944403792355 MSE: 1.5351809358318775


# predict

## get test data

In [14]:
test_data = pd.read_csv("PM25_test.csv")
test_data["Timestamp"] =pd.to_datetime(test_data["Date"].astype(str) + " " + test_data["Time"], format='%Y-%m-%d %H:%M:%S').astype(np.int64) 
test_data["Timestamp"] = test_data[ "Timestamp"] // TS_WEIGHT 


test_x = test_data
test_x.head()

,device_id,Date,Time,PM10,PM1,Temperature,Humidity,lat,lon,Timestamp
0,28C2DDDD415C,2017-01-31,00:04:05,0.0,0.0,18.62,92.0,23.741,120.755,1485821045
1,28C2DDDD415C,2017-01-31,00:10:01,0.0,0.0,18.75,92.0,23.741,120.755,1485821401
2,28C2DDDD415C,2017-01-31,00:15:57,0.0,0.0,18.62,92.0,23.741,120.755,1485821757
3,28C2DDDD415C,2017-01-31,00:21:53,0.0,0.0,18.62,92.0,23.741,120.755,1485822113
4,28C2DDDD415C,2017-01-31,00:27:48,0.0,0.0,18.62,92.0,23.741,120.755,1485822468


In [15]:
# add missing data
test_x = addMissingMean(test_x)
test_x.to_csv('test_x.csv', index  = False)
test_x = test_x.drop(['device_id','Date','Time'],axis=1)
test_x.head()

,PM10,PM1,Temperature,Humidity,lat,lon,Timestamp
0,37.640247,27.640247,18.62,92.0,23.741,120.755,1485821045
1,37.640247,27.640247,18.75,92.0,23.741,120.755,1485821401
2,37.640247,27.640247,18.62,92.0,23.741,120.755,1485821757
3,37.640247,27.640247,18.62,92.0,23.741,120.755,1485822113
4,37.640247,27.640247,18.62,92.0,23.741,120.755,1485822468


In [16]:
#predict
pre = reg.predict(test_x.values)
pre

array([ 53.25428782,  53.25431578,  53.25429665, ...,  29.46407547,
        27.19691141,  27.95675659])

In [17]:
result = test_data[["device_id"]]
result["pred_pm25"] = pre
result.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,device_id,pred_pm25
0,28C2DDDD415C,53.254288
1,28C2DDDD415C,53.254316
2,28C2DDDD415C,53.254297
3,28C2DDDD415C,53.254301
4,28C2DDDD415C,53.254305


In [18]:
#unique device id

pd_mean_pm25 = result.groupby(['device_id'])['pred_pm25'].mean().reset_index()
pd_mean_pm25.head()

,device_id,pred_pm25
0,28C2DDDD415C,53.258793
1,28C2DDDD47BC,59.905124
2,74DA3895C1F8,34.803673
3,74DA3895C1FA,32.517965
4,74DA3895C200,33.712204


In [19]:
#np.savetxt(, result.tolist(), delimiter = ',')
pd_mean_pm25.to_csv('l1-sumbmission.csv', index  = False)